<a href="https://colab.research.google.com/github/AngelTroncoso/SistemaDeRecomendacion/blob/main/SistemaDeRecomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configurar semilla para reproducibilidad
np.random.seed(42)
random.seed(42)

def generar_datos_cestas_compras(num_pedidos=1000, productos_por_cesta_promedio=4):
    """
    Genera datos ficticios con cestas de compras realistas para análisis de asociación

    Parámetros:
    num_pedidos (int): Número de pedidos únicos a generar
    productos_por_cesta_promedio (int): Promedio de productos por cesta

    Retorna:
    pd.DataFrame: DataFrame con registros donde múltiples productos pertenecen al mismo pedido
    """

    # Productos organizados por categorías con patrones de compra
    productos_data = {
        # Departamento 1: Alimentos
        1: {
            1: [  # Sección 1: Lácteos
                ("Leche Entera 1L", 2.50), ("Yogur Natural 500g", 3.20),
                ("Queso Manchego 200g", 8.90), ("Mantequilla 250g", 4.50),
                ("Leche Descremada 1L", 2.30), ("Yogur Griego 400g", 4.80),
                ("Crema de Leche 200ml", 2.80), ("Queso Fresco 300g", 5.20)
            ],
            2: [  # Sección 2: Carnes
                ("Pollo Entero kg", 5.20), ("Carne Molida kg", 12.50),
                ("Lomo de Cerdo kg", 15.80), ("Chuletas de Cordero kg", 22.90),
                ("Pechuga de Pollo kg", 8.70), ("Costillas de Res kg", 18.40),
                ("Salchichas kg", 6.80), ("Jamón Serrano 200g", 9.50)
            ],
            3: [  # Sección 3: Frutas y Verduras
                ("Manzanas Rojas kg", 3.40), ("Plátanos kg", 2.10),
                ("Tomates kg", 4.20), ("Cebollas kg", 1.80),
                ("Lechuga unidad", 1.50), ("Naranjas kg", 2.80),
                ("Zanahorias kg", 2.20), ("Papas kg", 1.90), ("Ajos kg", 4.50)
            ],
            10: [  # Sección 10: Panadería
                ("Pan Integral 500g", 2.80), ("Croissants 6 unidades", 4.50),
                ("Pan Blanco 600g", 2.20), ("Galletas Integrales 400g", 3.80),
                ("Tostadas 300g", 2.90), ("Pan de Molde 700g", 3.20)
            ]
        },
        # Departamento 2: Bebidas
        2: {
            4: [  # Sección 4: Refrescos
                ("Coca Cola 2L", 3.50), ("Pepsi 1.5L", 2.90),
                ("Fanta Naranja 2L", 3.20), ("Sprite 1.5L", 2.80),
                ("Agua Mineral 1.5L", 1.20), ("Jugo de Naranja 1L", 4.50),
                ("Agua Con Gas 1L", 1.80), ("Te Helado 1.5L", 3.10)
            ],
            5: [  # Sección 5: Alcohólicas
                ("Cerveza Lager 6pack", 8.90), ("Vino Tinto 750ml", 12.50),
                ("Whisky 750ml", 35.00), ("Vodka 750ml", 25.90),
                ("Cerveza Artesanal 500ml", 6.50), ("Vino Blanco 750ml", 11.80),
                ("Ron 750ml", 28.90), ("Cerveza Sin Alcohol 6pack", 7.20)
            ]
        },
        # Departamento 3: Limpieza
        3: {
            6: [  # Sección 6: Detergentes
                ("Detergente Líquido 1L", 4.80), ("Suavizante 900ml", 3.50),
                ("Jabón en Polvo 1kg", 5.20), ("Lavavajillas 750ml", 2.90),
                ("Desinfectante 1L", 3.80), ("Limpiador Multiuso 500ml", 2.60),
                ("Jabón de Manos 300ml", 2.40), ("Blanqueador 1L", 2.80)
            ],
            7: [  # Sección 7: Papel
                ("Papel Higiénico 12 rollos", 8.50), ("Toallas de Papel 6 rollos", 7.20),
                ("Servilletas 200 unidades", 2.40), ("Pañuelos Faciales 100 unidades", 1.80),
                ("Papel de Cocina 4 rollos", 5.60), ("Papel Aluminio 30m", 4.20),
                ("Bolsas de Basura 50 unidades", 3.90), ("Papel Encerado 25m", 3.60)
            ]
        },
        # Departamento 4: Cuidado Personal
        4: {
            8: [  # Sección 8: Higiene
                ("Champú 400ml", 6.50), ("Acondicionador 400ml", 6.80),
                ("Jabón Corporal 250ml", 4.20), ("Pasta Dental 100ml", 3.50),
                ("Desodorante 150ml", 4.80), ("Crema Hidratante 200ml", 7.90),
                ("Cepillo de Dientes", 2.80), ("Enjuague Bucal 500ml", 5.20)
            ],
            9: [  # Sección 9: Cosméticos
                ("Crema Facial 50ml", 15.90), ("Protector Solar 100ml", 12.50),
                ("Maquillaje Base 30ml", 18.90), ("Lápiz Labial", 8.50),
                ("Crema Antiarrugas 50ml", 25.80), ("Mascarilla Facial", 4.90),
                ("Removedor Maquillaje 200ml", 7.80), ("Serum Vitamina C 30ml", 22.50)
            ]
        }
    }

    # Definir patrones de compra (productos que suelen comprarse juntos)
    patrones_asociacion = {
        # Patrón: Desayuno
        "desayuno": [
            ("Leche Entera 1L", "Pan Integral 500g", "Mantequilla 250g"),
            ("Yogur Natural 500g", "Tostadas 300g"),
            ("Jugo de Naranja 1L", "Croissants 6 unidades"),
        ],
        # Patrón: Cena familiar
        "cena_familiar": [
            ("Pollo Entero kg", "Papas kg", "Zanahorias kg", "Cebollas kg"),
            ("Carne Molida kg", "Tomates kg", "Ajos kg", "Pasta"),
            ("Lomo de Cerdo kg", "Manzanas Rojas kg", "Vino Tinto 750ml"),
        ],
        # Patrón: Limpieza del hogar
        "limpieza": [
            ("Detergente Líquido 1L", "Suavizante 900ml", "Papel Higiénico 12 rollos"),
            ("Desinfectante 1L", "Toallas de Papel 6 rollos", "Limpiador Multiuso 500ml"),
            ("Jabón en Polvo 1kg", "Blanqueador 1L", "Bolsas de Basura 50 unidades"),
        ],
        # Patrón: Cuidado personal
        "cuidado_personal": [
            ("Champú 400ml", "Acondicionador 400ml", "Jabón Corporal 250ml"),
            ("Pasta Dental 100ml", "Cepillo de Dientes", "Enjuague Bucal 500ml"),
            ("Crema Facial 50ml", "Protector Solar 100ml", "Serum Vitamina C 30ml"),
        ],
        # Patrón: Fiesta/Reunión
        "fiesta": [
            ("Cerveza Lager 6pack", "Coca Cola 2L", "Servilletas 200 unidades"),
            ("Vino Tinto 750ml", "Vino Blanco 750ml", "Queso Manchego 200g"),
            ("Whisky 750ml", "Agua Mineral 1.5L"),
        ],
        # Patrón: Cocina básica
        "cocina_basica": [
            ("Aceite Oliva", "Sal", "Pimienta", "Ajos kg"),
            ("Tomates kg", "Cebollas kg", "Zanahorias kg"),
        ],
        # Patrón: Bebidas refrescantes
        "bebidas": [
            ("Agua Mineral 1.5L", "Jugo de Naranja 1L", "Te Helado 1.5L"),
            ("Coca Cola 2L", "Sprite 1.5L", "Agua Con Gas 1L"),
        ]
    }

    # Crear lista de todos los productos
    productos_list = []
    for dept_id, secciones in productos_data.items():
        for secc_id, productos in secciones.items():
            for nombre, precio in productos:
                productos_list.append({
                    'id_departamento': dept_id,
                    'id_seccion': secc_id,
                    'nombre_producto': nombre,
                    'precio_unitario': precio,
                    'id_producto': len(productos_list) + 1
                })

    # Crear un diccionario para búsqueda rápida de productos
    productos_dict = {p['nombre_producto']: p for p in productos_list}

    print(f"Generando {num_pedidos:,} pedidos con cestas de compras...")
    print(f"Productos disponibles: {len(productos_list)}")
    print(f"Patrones de asociación definidos: {len(patrones_asociacion)}")

    # Generar datos
    data = []
    fecha_inicio = datetime(2023, 1, 1)
    fecha_fin = datetime(2024, 12, 31)
    rango_dias = (fecha_fin - fecha_inicio).days

    num_clientes = min(num_pedidos // 4, 1000)  # Relación más realista cliente/pedidos

    registro_id = 1

    for pedido_id in range(1, num_pedidos + 1):
        if pedido_id % 5000 == 0:
            print(f"Progreso: {pedido_id:,}/{num_pedidos:,} pedidos")

        # Generar datos básicos del pedido
        dias_aleatorios = random.randint(0, rango_dias)
        fecha = fecha_inicio + timedelta(days=dias_aleatorios)
        hora = random.randint(800, 2200)
        id_cliente = random.randint(1, num_clientes)

        productos_cesta = []

        # Decidir tipo de compra (70% con patrones, 30% aleatoria)
        if random.random() < 0.7:
            # Compra con patrones de asociación
            num_patrones = random.choices([1, 2, 3], weights=[0.6, 0.3, 0.1])[0]
            patrones_elegidos = random.sample(list(patrones_asociacion.keys()),
                                            min(num_patrones, len(patrones_asociacion)))

            for patron in patrones_elegidos:
                # Elegir un grupo de productos del patrón
                grupo = random.choice(patrones_asociacion[patron])
                for nombre_producto in grupo:
                    if nombre_producto in productos_dict:
                        productos_cesta.append(productos_dict[nombre_producto])

            # Agregar algunos productos aleatorios adicionales
            productos_adicionales = random.randint(0, 3)
            for _ in range(productos_adicionales):
                producto_aleatorio = random.choice(productos_list)
                if producto_aleatorio not in productos_cesta:
                    productos_cesta.append(producto_aleatorio)

        else:
            # Compra completamente aleatoria
            num_productos = max(1, np.random.poisson(productos_por_cesta_promedio))
            productos_cesta = random.sample(productos_list,
                                          min(num_productos, len(productos_list)))

        # Asegurar que haya al menos un producto
        if not productos_cesta:
            productos_cesta = [random.choice(productos_list)]

        # Crear registros para cada producto en la cesta
        for producto in productos_cesta:
            # Generar cantidad realista
            if producto['precio_unitario'] > 50:  # Productos caros
                cantidad = random.choices([1, 2], weights=[0.85, 0.15])[0]
            elif producto['precio_unitario'] > 20:  # Productos medios
                cantidad = random.choices([1, 2, 3], weights=[0.7, 0.25, 0.05])[0]
            else:  # Productos baratos
                cantidad = random.choices([1, 2, 3, 4], weights=[0.5, 0.3, 0.15, 0.05])[0]

            precio_total = round(producto['precio_unitario'] * cantidad, 2)

            registro = {
                'id_pedido': pedido_id,
                'id_cliente': id_cliente,
                'fecha': fecha.strftime('%Y-%m-%d'),
                'hora': hora,
                'id_departamento': producto['id_departamento'],
                'id_seccion': producto['id_seccion'],
                'id_producto': producto['id_producto'],
                'nombre_producto': producto['nombre_producto'],
                'precio_unitario': producto['precio_unitario'],
                'cantidad': cantidad,
                'precio_total': precio_total
            }

            data.append(registro)
            registro_id += 1

    # Crear DataFrame
    df = pd.DataFrame(data)

    print(f"\n✅ Generación completada!")
    print(f"Total de registros: {len(df):,}")
    print(f"Total de pedidos únicos: {df['id_pedido'].nunique():,}")
    print(f"Promedio de productos por pedido: {len(df) / df['id_pedido'].nunique():.2f}")
    print(f"Rango de fechas: {df['fecha'].min()} a {df['fecha'].max()}")
    print(f"Clientes únicos: {df['id_cliente'].nunique():,}")
    print(f"Productos únicos: {df['id_producto'].nunique()}")
    print(f"Valor total de ventas: ${df['precio_total'].sum():,.2f}")

    return df

def analizar_cestas(df):
    """
    Función auxiliar para analizar las cestas de compras generadas
    """
    print("\n" + "="*60)
    print("ANÁLISIS DE CESTAS DE COMPRAS")
    print("="*60)

    # Agrupar productos por pedido
    cestas = df.groupby('id_pedido')['nombre_producto'].apply(lambda x: ', '.join(x))

    print(f"📊 Distribución de tamaños de cesta:")
    tamaños_cesta = df.groupby('id_pedido').size()
    print(tamaños_cesta.value_counts().sort_index().head(10))

    print(f"\n🛒 Ejemplos de cestas de compras:")
    for i, (pedido_id, productos) in enumerate(cestas.head(10).items()):
        print(f"Pedido {pedido_id}: {productos}")

    print(f"\n📈 Productos más frecuentes:")
    productos_freq = df['nombre_producto'].value_counts().head(10)
    for producto, freq in productos_freq.items():
        print(f"{producto}: {freq} veces")

    return cestas

# Ejemplo de uso
if __name__ == "__main__":
    # Generar datos con cestas realistas
    df_pedidos = generar_datos_cestas_compras(num_pedidos=10000, productos_por_cesta_promedio=4)

    # Guardar datos
    df_pedidos.to_csv('cestas_compras_ficticias.csv', index=False)

Generando 10,000 pedidos con cestas de compras...
Productos disponibles: 79
Patrones de asociación definidos: 7
Progreso: 5,000/10,000 pedidos
Progreso: 10,000/10,000 pedidos

✅ Generación completada!
Total de registros: 50,862
Total de pedidos únicos: 10,000
Promedio de productos por pedido: 5.09
Rango de fechas: 2023-01-01 a 2024-12-31
Clientes únicos: 1,000
Productos únicos: 79
Valor total de ventas: $540,747.90


In [3]:
df  = pd.read_csv('cestas_compras_ficticias.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50862 entries, 0 to 50861
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_pedido        50862 non-null  int64  
 1   id_cliente       50862 non-null  int64  
 2   fecha            50862 non-null  object 
 3   hora             50862 non-null  int64  
 4   id_departamento  50862 non-null  int64  
 5   id_seccion       50862 non-null  int64  
 6   id_producto      50862 non-null  int64  
 7   nombre_producto  50862 non-null  object 
 8   precio_unitario  50862 non-null  float64
 9   cantidad         50862 non-null  int64  
 10  precio_total     50862 non-null  float64
dtypes: float64(2), int64(7), object(2)
memory usage: 4.3+ MB


In [5]:
df['fecha'] = pd.to_datetime(df['fecha'])

In [6]:
df_cesta = df[['id_pedido','nombre_producto']]
df_cesta

,id_pedido,nombre_producto
0,1,Coca Cola 2L
1,1,Galletas Integrales 400g
2,1,Plátanos kg
3,1,Costillas de Res kg
4,1,Cepillo de Dientes
...,...,...
50857,10000,Bolsas de Basura 50 unidades
50858,10000,Protector Solar 100ml
50859,10000,Jugo de Naranja 1L
50860,10000,Jabón en Polvo 1kg


In [7]:
# Agrupar los productos por id_pedido
df_agrupado = df_cesta.groupby('id_pedido')['nombre_producto'].apply(lambda producto: ','.join(producto))
df_agrupado

,nombre_producto
id_pedido,
1,"Coca Cola 2L,Galletas Integrales 400g,Plátanos..."
2,"Cepillo de Dientes,Jabón de Manos 300ml,Gallet..."
3,"Coca Cola 2L,Sprite 1.5L,Agua Con Gas 1L"
4,"Yogur Natural 500g,Tostadas 300g"
5,"Ajos kg,Lomo de Cerdo kg,Tostadas 300g"
...,...
9996,"Desinfectante 1L,Toallas de Papel 6 rollos,Lim..."
9997,"Pollo Entero kg,Papas kg,Zanahorias kg,Cebolla..."
9998,"Ajos kg,Papel Higiénico 12 rollos,Lápiz Labial..."


In [8]:
# Aplicar pd.get_dummies() para transformar los productos en columnas con 0/1
df_transacciones = df_agrupado.str.get_dummies(sep=',')
df_transacciones

,Acondicionador 400ml,Agua Con Gas 1L,Agua Mineral 1.5L,Ajos kg,Blanqueador 1L,Bolsas de Basura 50 unidades,Carne Molida kg,Cebollas kg,Cepillo de Dientes,Cerveza Artesanal 500ml,...,Toallas de Papel 6 rollos,Tomates kg,Tostadas 300g,Vino Blanco 750ml,Vino Tinto 750ml,Vodka 750ml,Whisky 750ml,Yogur Griego 400g,Yogur Natural 500g,Zanahorias kg
id_pedido,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
9998,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Soporte para cada producto
soporte = df_transacciones.mean() * 100
soporte.sort_values(ascending=False)

,0
Jugo de Naranja 1L,15.02
Coca Cola 2L,14.95
Agua Mineral 1.5L,14.94
Cebollas kg,14.93
Tomates kg,14.85
...,...
Naranjas kg,2.65
Pan Blanco 600g,2.64
Lavavajillas 750ml,2.61
Crema Antiarrugas 50ml,2.57


In [10]:
# Función para calcular la confianza entre dos productos en la muestra
def confianza(antecedente, consecuente):
    # Casos donde se compraron ambos productos
    conjunto_ac = df_transacciones[(df_transacciones[antecedente] == 1) &
                                   (df_transacciones[consecuente] == 1)]
    # Confianza = compras conjuntas / compras de producto A
    return len(conjunto_ac) / df_transacciones[antecedente].sum()

In [11]:
# Función para calcular el lift entre dos productos en la muestra
def lift(antecedente, consecuente):
    soporte_a = df_transacciones[antecedente].mean()
    soporte_c = df_transacciones[consecuente].mean()
    conteo_ac = len(df_transacciones[(df_transacciones[antecedente] == 1) &
                                   (df_transacciones[consecuente] == 1)])
    soporte_ac = conteo_ac / len(df_transacciones)
    return soporte_ac / (soporte_a * soporte_c)

In [12]:
from itertools import combinations

# Definir un umbral para la confianza mínima
umbral_confianza = 0.05
asociaciones = []

# Generar combinaciones de productos y calcular confianza y lift
for antecedente, consecuente in combinations(df_transacciones.columns, 2):

    # Soporte del antecedente
    soporte_a = df_transacciones[antecedente].mean()

    # Calcular confianza
    conf = confianza(antecedente, consecuente)
    if conf > umbral_confianza:
        asociaciones.append({
            'antecedente': antecedente,
            'consecuente': consecuente,
            'soporte_a': round(soporte_a * 100,1),
            'confianza': round(conf * 100,1),
            'lift': round(lift(antecedente, consecuente),1)
        })


# Convertir las asociaciones en un DataFrame
df_asociaciones = pd.DataFrame(asociaciones)

# Ordenar las asociaciones por confianza de mayor a menor
df_asociaciones.sort_values(by='lift', ascending=False, inplace=True)

df_asociaciones

,antecedente,consecuente,soporte_a,confianza,lift
1121,Mantequilla 250g,Pan Integral 500g,7.9,68.2,9.0
156,Blanqueador 1L,Bolsas de Basura 50 unidades,7.1,65.8,9.0
1475,Queso Manchego 200g,Vino Blanco 750ml,7.4,67.8,8.9
167,Blanqueador 1L,Jabón en Polvo 1kg,7.1,65.9,8.9
1079,Lomo de Cerdo kg,Manzanas Rojas kg,7.7,65.2,8.8
...,...,...,...,...,...
53,Agua Con Gas 1L,Jugo de Naranja 1L,9.9,7.3,0.5
380,Cerveza Lager 6pack,Vino Tinto 750ml,7.7,6.5,0.5
1114,Lápiz Labial,Tomates kg,2.9,8.0,0.5
37,Agua Con Gas 1L,Agua Mineral 1.5L,9.9,7.6,0.5


**Generamos una tabla de los productos únicos con su 'id_producto', 'id_seccion', 'id_departamento' para enriquecer la tabla de regla**

In [13]:
# Crear una tabla con los productos únicos y las columnas correspondientes
productos_unicos = df[['id_producto', 'id_seccion', 'id_departamento', 'nombre_producto']].drop_duplicates()

productos_unicos

,id_producto,id_seccion,id_departamento,nombre_producto
0,32,4,2,Coca Cola 2L
1,29,10,1,Galletas Integrales 400g
2,18,3,1,Plátanos kg
3,14,2,1,Costillas de Res kg
4,70,8,4,Cepillo de Dientes
...,...,...,...,...
373,46,5,2,Ron 750ml
376,33,4,2,Pepsi 1.5L
448,7,1,1,Crema de Leche 200ml
459,68,8,4,Desodorante 150ml


In [14]:
df_asociaciones_enriquecido = df_asociaciones.merge(productos_unicos, left_on='antecedente', right_on='nombre_producto', how='left').drop(columns=['nombre_producto'])
df_asociaciones_enriquecido.columns = ['antecedente', 'consecuente', 'soporte_a', 'confianza', 'lift', 'id_producto_a', 'id_seccion_a', 'id_departamento_a']
df_asociaciones_enriquecido

,antecedente,consecuente,soporte_a,confianza,lift,id_producto_a,id_seccion_a,id_departamento_a
0,Mantequilla 250g,Pan Integral 500g,7.9,68.2,9.0,4,1,1
1,Blanqueador 1L,Bolsas de Basura 50 unidades,7.1,65.8,9.0,55,6,3
2,Queso Manchego 200g,Vino Blanco 750ml,7.4,67.8,8.9,3,1,1
3,Blanqueador 1L,Jabón en Polvo 1kg,7.1,65.9,8.9,55,6,3
4,Lomo de Cerdo kg,Manzanas Rojas kg,7.7,65.2,8.8,11,2,1
...,...,...,...,...,...,...,...,...
1588,Agua Con Gas 1L,Jugo de Naranja 1L,9.9,7.3,0.5,38,4,2
1589,Cerveza Lager 6pack,Vino Tinto 750ml,7.7,6.5,0.5,40,5,2
1590,Lápiz Labial,Tomates kg,2.9,8.0,0.5,75,9,4
1591,Agua Con Gas 1L,Agua Mineral 1.5L,9.9,7.6,0.5,38,4,2


In [15]:
#Descargar archivo
df_asociaciones_enriquecido.to_csv('reglas.csv', index=False, sep=';', decimal=',')